## 22. 딕셔너리와 튜플보다는 헬퍼 클래스로 관리하자

* 객체의 수명이 지속되는 동안 동적인 내부 상태를 관리하기
    - '동적': 예상하지 못한 식별자들을 관리해야 하는 상황
        - 예: 이름을 모르는 학생 집단의 성적을 기록

### I. 방법: 딕셔너리와 튜플로 관리

* 장점
    - 객체의 수명이 지속되는 동안 동적인 내부 상태를 관리하는 용도로 아주 좋음

In [1]:
# 클래스 정의
# 학생별로 미리 정의된 속성을 사용하지 않고 딕셔너리에 이름 저장
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [2]:
# 클래스 사용
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
print(book.average_grade('Isaac Newton'))

90.0


* 단점
    - 과도하게 사용하면 코드가 취약해질 수 있음
    - 여러 단계의 딕셔너리를 처리하게 되면 복잡해짐

In [5]:
# 클래스 확장
# 성적을 과목별로 저장
# _grades 딕셔너리를 변경해서 학생 이름(키)을 다른 딕셔너리(값)에 매핑
# 가장 안쪽 딕셔너리는 과목(키)을 성적(값)에 매핑
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
    
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append(grade)
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [7]:
# 클래스 사용
book = BySubjectGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75)
book.report_grade('Albert Einstein', 'Math', 65)
book.report_grade('Albert Einstein', 'Gym', 90)
book.report_grade('Albert Einstein', 'Gym', 95)
print(book.average_grade('Albert Einstein'))

81.25


In [8]:
# 클래스 변경
# 시험 종류에 따른 가중치 적용
# 가장 안쪽 딕셔너리를 변경해 과목(키)를 성적과 비중을 담은 튜플에 매핑
class WeightedGradebook(BySubjectGradebook):
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            score_sum += subject_avg / total_weight
            score_count += 1
        return score_sum / score_count

In [12]:
# 클래스 사용
book = WeightedGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75, 0.5)
book.report_grade('Albert Einstein', 'Math', 65, 0.5)
book.report_grade('Albert Einstein', 'Gym', 90, 0.5)
book.report_grade('Albert Einstein', 'Gym', 95, 0.5)
print(book.average_grade('Albert Einstein'))

81.25


### II. 더 나은 방법: 헬퍼 클래스로 관리

In [13]:
grades = []
grades.append((95, 0.45))
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

In [15]:
grades = []
grades.append((95, 0.45, 'Great job'))
total = sum(score * weight for score, weight, _ in grades)
total_weight = sum(weight for _, weight, _ in grades)
average_grade = total / total_weight

In [16]:
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

In [17]:
class Subject(object):
    def __init__(self):
        self._grades = []
    
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [19]:
class Student(object):
    def __init__(self):
        self._subjects = {}
    
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [20]:
class Gradebook(object):
    def __init__(self):
        self._students = {}
        
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [21]:
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(80, 0.10)
print(albert.average_grade())

80.0


## 23.

In [22]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


In [23]:
def log_missing():
    print('Key added')
    return 0

In [24]:
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

In [26]:
result = collections.defaultdict(log_missing, current)
print('Before: ', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before:  {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


In [27]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count # closure
        added_count += 1
        return 0
    
    result = collections.defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count

In [28]:
result, count = increment_with_report(current, increments)
assert count == 2

In [29]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

In [30]:
counter = CountMissing()
result = collections.defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount
assert counter.added == 2

In [31]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0

In [32]:
counter = BetterCountMissing()
counter()
assert callable(counter)

In [33]:
counter = BetterCountMissing()
result = collections.defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2